Each project is going to be either an Expansion or a Replacement project with an associated set of assumptions regarding (1) the initial outlay, (2) annual after-tax operating cash flows, and (3) terminal year after-tax non-operating cash flows. 

The formulaic construction of each of these is relatively straight-forward, however, it is their construction and subsequent testing that is of primary interest. 

Therefore, we need to consider how exactly our series are constructed from the bottom up. 

The most basic unit would be to construct a simple list that is passed to the appropriate class as follows:

# Expansion

In [71]:
import pandas as pd
import numpy as np

In [18]:
'''
The initial investment outlays are structured as (1) investments in new fixed capital and (2) investment in net
working capital. 

For example, let's say that our investment includes a $200,000 outlay which includes $25,000 for nondepreciable 
land, plus $175,000 for equipment that will be depreciated straight-line to zero over five years. 
'''

non_depreciable = 25000.00

depreciable = 175000.00

economic_life = 5.00

In [19]:
'''
The investment in net working capital is the net investment in short-term assets required for the investment. 

This is the investment in receivables and inventory needed, less teh short-term payables generated by 
the project. 

In this case, the project required $50,000 of current assets but generated $20,000 in current liabilities, 
resulting in a total investment in net working capital of $30,000.
'''

NWCInv = 30000.00

In [20]:
'''
Now, let's assume that each year, sales will be $220,000 and cash operating expenses will be $90,000. 

Annual depreciation for the $175,000 depreciable equipment is $35,000 (one-fifth of the cost). 

The result is an operating income before taxes of $95,000.

Income taxes at a 40 percent rate are 0.40 * $95,000 = $38,000. 

This leaves operating income after taxes of $57,000. 

Adding back the depreciation charge of $35,000 gives the annual after-tax operating cash flow of $92,000. 
'''

S = 220000.00
C = 90000.00
T = 0.40

In [21]:
'''
Finally, we turn to the terminal year non-operating cash flow.

At the end of year 5, the company will sell off the fixed capital assets. 

In this case, the fixed capital assets (including the land) are sold for $50,000, which represents a gain of 
$25,000 over the remaining book value of $25,000. 

The gain of $25,000 is taxed at 40 percent, resulting in a tax of $10,000. 

This leaves $40,000 for the fixed capital assets after taxes. 

Additionally, teh net working capital investment of $30,000 is recovered, as the short-term asssets (such as
inventory and receivables) and short-term liabilities (such as payables) are no longer needed for the project. 

Total terminal year non-operating cash flows are then $70,000. 
'''

Sal = 50000.00

In [32]:
'''
Additionally, the investment project has a required rate of return of 10 percent. 

Discounting the future cash flows at 10 percent and subtracting the investment outlay gives us our NPV. 
'''

r = 0.10

In [87]:
class Expansion:
    
    def __init__(self, depreciable, non_depreciable, economic_life, NWCInv, S, C, T, Sal):
        self.economic_life = economic_life
        self.Outlay = (depreciable + non_depreciable) + NWCInv
        self.CF = (S - C - (depreciable / self.economic_life))*(1 - T) + (depreciable / self.economic_life)
        self.TNOCF = Sal + NWCInv - T*(Sal - non_depreciable)
    
    def NPV(self, r):
        CFs = 0
        for x in [x for x in range(1, int(self.economic_life) + 1)]:
            CFs += (self.CF / ((1 + r)**x))
            
        NPV = -self.Outlay + CFs + (self.TNOCF / ((1 + r)**self.economic_life))
        
        return round(NPV)
    
    def IRR(self):
        return round(np.irr([-self.Outlay] + [self.CF]*(int(self.economic_life)-1) + [self.TNOCF]), 4)

In [88]:
project = Expansion(depreciable, non_depreciable, economic_life, NWCInv, S, C, T, Sal)

In [89]:
project.Outlay

230000.0

In [90]:
project.CF

92000.0

In [91]:
project.TNOCF

70000.0

In [92]:
project.NPV(r)

162217

In [93]:
project.IRR()

0.2721

# Replacement

The only major difference here is that Sal_init and B_init are included and these relate to the salvage value at
initiation of the old equipment being replaced and it's corresponding book value for calculating taxes. 

In [94]:
class Replacement:
    
    def __init__(self, Sal_init, B_init, depreciable, non_depreciable, economic_life, NWCInv, S, C, T, Sal):
        self.economic_life = economic_life
        self.Outlay = (depreciable + non_depreciable) + NWCInv - Sal_init + T*(Sal_init - B_init)
        self.CF = (S - C - (depreciable / self.economic_life))*(1 - T) + (depreciable / self.economic_life)
        self.TNOCF = Sal + NWCInv - T*(Sal - non_depreciable)
    
    def NPV(self, r):
        CFs = 0
        for x in [x for x in range(1, int(self.economic_life) + 1)]:
            CFs += (self.CF / ((1 + r)**x))
            
        NPV = -self.Outlay + CFs + (self.TNOCF / ((1 + r)**self.economic_life))
        
        return round(NPV)
    
    def IRR(self):
        return round(np.irr([-self.Outlay] + [self.CF]*(int(self.economic_life)-1) + [self.TNOCF]), 4)

In [ ]:
class Compare:
    
    def __init__(self, project1, project2, exclusive=True):
        self.project1 = project1
        self.project2 = project2